In [18]:
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import math

In [19]:
sf = gpd.read_file("../data/raw/taxi_zones/taxi_zones.shp")
zones = pd.read_csv("../data/raw/taxi_zones/taxi+_zone_lookup.csv")

sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14..."


In [20]:
# LR r-square table
# read in data required
df1 = pd.read_csv('../data/curated/LR_test_prediction.csv')
df2 = pd.read_csv('../data/curated/y_test')
df3 = pd.read_csv('../data/curated/X_test')

In [21]:
# merge three dataframes read in into one dataframe that includes real y test values, predicted y test values
# and PULocationID
lr_test = pd.merge(df1['0'],df2['income_rate'],left_index=True, right_index=True)
lr_test = pd.merge(lr_test,df3['PULocationID'],left_index=True, right_index=True)
lr_test = lr_test.rename(columns={'0': 'predicted_income_rate'})

In [22]:
# calculated the r squared value by locationID
lr_rsq = []
for i in range(1, 264):
    sub_df = lr_test[lr_test['PULocationID'] == i]
    if len(sub_df) > 0:
        mean_testing = np.mean(sub_df['income_rate'])
        sub_df['Null_SS'] = (sub_df['income_rate'] - mean_testing)**2
        sub_df['model_SS'] = (sub_df['income_rate'] - sub_df['predicted_income_rate'])**2

        SSR_test = sum(sub_df['model_SS'])/len(sub_df)
        SST_test = sum(sub_df['Null_SS'])/len(sub_df)
        if SST_test > 0:
            R2_test = 1-SSR_test/SST_test
            R2_test
            lr_rsq.append(R2_test)
        else:
            lr_rsq.append(0)
    else:
        lr_rsq.append(0)

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [23]:
# sort the calculataed locationID r squared value into dataframe
lr_map_data = pd.DataFrame({'PULocationID': range(1, 264), 'r2': lr_rsq})
lr_map_data

,PULocationID,r2
0,1,-0.065398
1,2,-21.271172
2,3,-7.852920
3,4,-1800.636763
4,5,-3602.069537
...,...,...
258,259,-11.708484
259,260,-27.446619
260,261,-1422.253969
261,262,-976.283959


In [24]:
lr_map_data.loc[lr_map_data['r2'] <0, 'r2'] = 0

In [25]:
# RFR r-square table
# read in required data
df4 = pd.read_csv('../data/curated/RFR_test_prediction.csv')

In [26]:
# merge three dataframes read in into one dataframe that includes real y test values, predicted y test values
# and PULocationID for random forest regeression
rfr_test = pd.merge(df4['predicted_income_rate'],df2['income_rate'],left_index=True, right_index=True)
rfr_test = pd.merge(rfr_test,df3['PULocationID'],left_index=True, right_index=True)

In [27]:
# calculated the r squared value by locationID
rfr_rsq = []
for i in range(1, 264):
    sub_df = rfr_test[rfr_test['PULocationID'] == i]
    if len(sub_df) > 0:
        mean_testing = np.mean(sub_df['income_rate'])
        sub_df['Null_SS'] = (sub_df['income_rate'] - mean_testing)**2
        sub_df['model_SS'] = (sub_df['income_rate'] - sub_df['predicted_income_rate'])**2

        SSR_test = sum(sub_df['model_SS'])/len(sub_df)
        SST_test = sum(sub_df['Null_SS'])/len(sub_df)
        if SST_test > 0:
            R2_test = 1-SSR_test/SST_test
            R2_test
            rfr_rsq.append(R2_test)
        else:
            rfr_rsq.append(0)
    else:
        rfr_rsq.append(0)

/opt/miniconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [28]:
# sort the calculataed locationID r squared value into dataframe
rfr_map_data = pd.DataFrame({'PULocationID': range(1, 264), 'r2': rfr_rsq})
rfr_map_data

,PULocationID,r2
0,1,-8.649149e-01
1,2,-1.001231e+04
2,3,-2.895216e+03
3,4,-7.214692e+05
4,5,-2.639487e+06
...,...,...
258,259,-7.931324e+03
259,260,-1.933972e+04
260,261,-6.081325e+05
261,262,-4.866332e+05


In [29]:
rfr_map_data.loc[rfr_map_data['r2'] <0, 'r2'] = 0

In [30]:
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((-74.18445 40.69500, -74.18449 40.695..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((-73.82338 40.63899, -73.82277 ..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((-73.84793 40.87134, -73.84725 40.870..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((-73.97177 40.72582, -73.97179 40.725..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((-74.17422 40.56257, -74.17349 40.562..."


In [ ]:
# create map for accuracy (for linear regression)
lr_gdf = gpd.GeoDataFrame(pd.merge(lr_map_data, sf,
                                       left_on='PULocationID',
                                       right_on='LocationID',
                                       how='right'))
lr_gdf

In [ ]:
geoJSON = lr_gdf[['LocationID','geometry']]
lr_map = folium.Map(location=[40.66, -73.94],tiles="Stamen Terrain", zoom_start=10)

lr_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
    data = lr_gdf,
    columns=['PULocationID', 'r2'],
    key_on = 'properties.LocationID',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name = 'R2'
))

In [ ]:
# create map for accuracy (for random forest regression)
rfr_gdf = gpd.GeoDataFrame(pd.merge(rfr_map_data, sf,
                                       left_on='PULocationID',
                                       right_on='LocationID',
                                       how='right'))
rfr_gdf

In [ ]:
geoJSON = rfr_gdf[['LocationID','geometry']]
rfr_map = folium.Map(location=[40.66, -73.94],tiles="Stamen Terrain", zoom_start=10)

rfr_map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
    data = rfr_gdf,
    columns=['PULocationID', 'r2'],
    key_on = 'properties.LocationID',
    fill_color='YlOrRd',
    nan_fill_color='black',
    legend_name = 'R2'
))